<a href="https://colab.research.google.com/github/devdebdeb/ML-Classificacao/blob/main/Combinando_classificadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlxtend
!pip install catboost

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier, BaggingClassifier, ExtraTreesClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.evaluate import confusion_matrix, scoring
import joblib

#Preprocessing data

In [ ]:
url1 = 'https://raw.githubusercontent.com/alura-cursos/combina-classificadores/main/dados/train.csv'
url2 = 'https://raw.githubusercontent.com/alura-cursos/combina-classificadores/main/dados/test.csv'

In [ ]:
train = pd.read_csv(url1)
test = pd.read_csv(url2)

In [ ]:
train.head(1)

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied


Problema de classificação binária

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

In [ ]:
train.isnull().sum()

,0
Unnamed: 0,0
id,0
Gender,0
Customer Type,0
Age,0
Type of Travel,0
Class,0
Flight Distance,0
Inflight wifi service,0
Departure/Arrival time convenient,0


In [ ]:
def preprocesser(df):
    df.drop(['Unnamed: 0', 'id'], axis=1, inplace=True)
    df.dropna(inplace=True)
    #categorical_columns = [column for column in df.columns if df[column].dtype=='object']
    categorical_columns = ['Gender', 'Customer Type', 'Class', 'Type of Travel']
    df = pd.get_dummies(df, columns=categorical_columns)
    X = df.drop('satisfaction', axis=1)
    y = df['satisfaction']
    return X, y

In [ ]:
X_train, y_train = preprocesser(train)
X_test, y_test = preprocesser(test)

# Testando modelos clássicos

## Tree-Based Testing

In [ ]:
#cv1 para testar o desempenho do modelo tree1
model1 = DecisionTreeClassifier(random_state=42)
cv = cross_validate(model1, X_train, y_train, cv=5)

In [ ]:
cv['test_score'].mean()

0.944176314621885

In [ ]:
model1.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [ ]:
y_pred = model1.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.9473989109025606

## Pipeline testing

In [ ]:
model2 = LogisticRegression(random_state=42)
cv = cross_validate(model2, X_train, y_train)
#Atingiu o limite de iterações == Problema de convergência de algorítmo

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [ ]:
cv['test_score'].mean()

0.8195358622181667

In [ ]:
pipeline = Pipeline([
    ('Scaler', StandardScaler()),
    ('model', model2)
])

In [ ]:
cv = cross_validate(pipeline, X_train, y_train, cv=5)

In [ ]:
cv['test_score'].mean()

0.8749541859894492

# Votação

Tentar combinar os dois modelos para obter um melhor resultado

## Hard Voting

In [ ]:
model3 = GaussianNB()

In [ ]:
model_names = ['Árvores', 'Logística', 'Naive Bayes']

In [ ]:
pipelines = []

for model, name in zip([model1, model2, model3], model_names):
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    pipelines.append(pipeline)

    cv = cross_validate(pipeline, X_train, y_train, cv=5)
    print(cv['test_score'].mean())

0.9441956219707108
0.8749541859894492
0.8483406855152964


In [ ]:
voting = VotingClassifier(estimators=[
    (model_names[0], pipelines[0]),
    (model_names[1], pipelines[1]),
    (model_names[2], pipelines[2])
], voting='hard')

In [ ]:
voting

VotingClassifier(estimators=[('Árvores',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model',
                                               DecisionTreeClassifier(random_state=42))])),
                             ('Logística',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model',
                                               LogisticRegression(random_state=42))])),
                             ('Naive Bayes',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model', GaussianNB())]))])

In [ ]:
cv = cross_validate(voting, X_train, y_train, cv=5)

In [ ]:
cv['test_score'].mean()

0.8916733054751944

## Soft Voting

In [ ]:
voting = VotingClassifier(estimators=[
    (model_names[0], pipelines[0]),
    (model_names[1], pipelines[1]),
    (model_names[2], pipelines[2])
], voting='soft')

In [ ]:
cv = cross_validate(voting, X_train, y_train, cv=5)

In [ ]:
cv['test_score'].mean()

0.900621712100811

[0 , 1, 0] -> hard - 0

[0.8 * 0 ,0.1 * 1, 0.1* 0] -> soft - 1


In [ ]:
params = {
    'voting':['hard', 'soft'],
    'weights': [(1,1,1),(2,1,1),(1,2,1),(1,1,2)]
}

In [ ]:
gridsearch = GridSearchCV(voting, params, n_jobs=-1)

In [ ]:
gridsearch.fit(X_train, y_train)

GridSearchCV(estimator=VotingClassifier(estimators=[('Árvores',
                                                     Pipeline(steps=[('scaler',
                                                                      StandardScaler()),
                                                                     ('model',
                                                                      DecisionTreeClassifier(random_state=42))])),
                                                    ('Logística',
                                                     Pipeline(steps=[('scaler',
                                                                      StandardScaler()),
                                                                     ('model',
                                                                      LogisticRegression(random_state=42))])),
                                                    ('Naive Bayes',
                                                     Pipeline(steps=[('scaler',
                                                                      StandardScaler()),
                                                                     ('model',
                                                                      GaussianNB())]))],
                                        voting='soft'),
             n_jobs=-1,
             param_grid={'voting': ['hard', 'soft'],
                         'weights': [(1, 1, 1), (2, 1, 1), (1, 2, 1),
                                     (1, 1, 2)]})

In [ ]:
gridsearch.best_params_

{'voting': 'soft', 'weights': (2, 1, 1)}

In [ ]:
gridsearch.best_score_

0.9441956219707108

In [ ]:
y_pred = gridsearch.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.9473216699494071

# Bagging

## BaggingClassifier

In [ ]:
bagging_classifier = BaggingClassifier(n_estimators=10, random_state=42)

In [ ]:
bagging_classifier.fit(X_train, y_train)

BaggingClassifier(random_state=42)

In [ ]:
y_pred = bagging_classifier.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.9588305719692581

## Validando o Bagging

In [ ]:
base_model = pipelines[0]

In [ ]:
params_grid = {
    'n_estimators': [10, 20, 30],
    'max_samples': [0.5, 0.7, 0.9],
    'max_features': [0.5, 0.7, 0.9]
}

In [ ]:
bagging_grid = GridSearchCV(
    BaggingClassifier(),
    params_grid,
    cv=5,
    n_jobs=-1
)

In [ ]:
bagging_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=BaggingClassifier(), n_jobs=-1,
             param_grid={'max_features': [0.5, 0.7, 0.9],
                         'max_samples': [0.5, 0.7, 0.9],
                         'n_estimators': [10, 20, 30]})

In [ ]:
best_params = bagging_grid.best_params_

In [ ]:
bagging_classifier = BaggingClassifier(estimator=base_model, **best_params)

In [ ]:
bagging_classifier.fit(X_train, y_train)

BaggingClassifier(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                            ('model',
                                             DecisionTreeClassifier(random_state=42))]),
                  max_features=0.9, max_samples=0.9, n_estimators=30)

In [ ]:
y_pred = bagging_classifier.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.9623450353377361

## ExtraTreesClassifier

In [ ]:
params_grid = {
    'n_estimators': [10, 20, 30],
    'max_features': [0.5, 0.7, 0.9]
}

In [ ]:
extratrees_grid = GridSearchCV(
    ExtraTreesClassifier(),
    params_grid,
    cv=5,
    n_jobs=-1
)

In [ ]:
extratrees_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=ExtraTreesClassifier(), n_jobs=-1,
             param_grid={'max_features': [0.5, 0.7, 0.9],
                         'n_estimators': [10, 20, 30]})

In [ ]:
param_trees = extratrees_grid.best_params_

In [ ]:
extratrees_classifier = ExtraTreesClassifier(**best_params)

In [ ]:
extratrees_classifier.fit(X_train, y_train)

ExtraTreesClassifier(max_features=0.5, n_estimators=30)

In [ ]:
y_pred = extratrees_classifier.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.9631946858224231

In [ ]:
extratrees_grid.best_score_

0.9629515304753564

# Boosting

## AdaBoost

In [ ]:
ada_model = AdaBoostClassifier(n_estimators=50, learning_rate=1)

In [ ]:
ada_model.fit(X_train, y_train)

AdaBoostClassifier(learning_rate=1)

In [ ]:
y_pred = ada_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.9263121306916927

### GridSearch - AdaBoost

In [ ]:
ada_model2 = AdaBoostClassifier(n_estimators=50, learning_rate=1)

In [ ]:
param_grid={
'n_estimators': [50,10,200],
'learning_rate': [0.1,0.01,0.001]
}

In [ ]:
grid_search = GridSearchCV(ada_model2, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=AdaBoostClassifier(learning_rate=1), n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'n_estimators': [50, 10, 200]},
             scoring='accuracy')

In [ ]:
y_pred = grid_search.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.9248445525817789

In [ ]:
grid_search.best_score_

0.9258161725228913

## CatBoostClassifier

In [ ]:
param_grid= {
'iterations': [100,200,300],
'depth': [4,6,8],
'learning_rate': [0.1,0.01,0.001]
}

In [ ]:
grid_search = GridSearchCV(estimator=CatBoostClassifier(verbose=0), param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x7fbf26f4edd0>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 8], 'iterations': [100, 200, 300],
                         'learning_rate': [0.1, 0.01, 0.001]},
             scoring='accuracy')

In [ ]:
y_pred = grid_search.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.9638898544008033

In [ ]:
cat_params = grid_search.best_params_

# Stacking

Método de combinação de modelos, tal como o voting

## StackingClassifier

In [ ]:
base_models= [
    (model_names[0], pipelines[0]),
    (model_names[1], pipelines[1]),
    (model_names[2], pipelines[2])
]

In [ ]:
base_models

[('Árvores',
  Pipeline(steps=[('scaler', StandardScaler()),
                  ('model', DecisionTreeClassifier(random_state=42))])),
 ('Logística',
  Pipeline(steps=[('scaler', StandardScaler()),
                  ('model', LogisticRegression(random_state=42))])),
 ('Naive Bayes',
  Pipeline(steps=[('scaler', StandardScaler()), ('model', GaussianNB())]))]

In [ ]:
meta_model = LogisticRegression(random_state=42)

In [ ]:
stacking_classifier = StackingClassifier(estimators=base_models, final_estimator=meta_model)

In [ ]:
stacking_classifier.fit(X_train, y_train)

StackingClassifier(estimators=[('Árvores',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('model',
                                                 DecisionTreeClassifier(random_state=42))])),
                               ('Logística',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('model',
                                                 LogisticRegression(random_state=42))])),
                               ('Naive Bayes',
                                Pipeline(steps=[('scaler', StandardScaler()),
                                                ('model', GaussianNB())]))],
                   final_estimator=LogisticRegression(random_state=42))

In [ ]:
y_pred = stacking_classifier.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.9473216699494071

## CV

In [ ]:
cat_params = grid_search.best_params_

In [ ]:
param_trees = extratrees_grid.best_params_

In [ ]:
base_models = [
    CatBoostClassifier(**cat_params),
    ExtraTreesClassifier(**param_trees),
    pipelines[2]
]

In [ ]:
stacking_cv_classifier = StackingCVClassifier(classifiers=base_models, meta_classifier=meta_model)

In [ ]:
mapping = {'neutral or dissatisfied':0, 'satisfied':1}

In [ ]:
y_train, y_test = y_train.replace(mapping), y_test.replace(mapping)

In [ ]:
stacking_cv_classifier.fit(X_train, y_train)

In [ ]:
y_pred = stacking_cv_classifier.predict(X_test)

In [ ]:
stacking_cv_classifier.score(X_test, y_test)

0.9634650291584598

##Result Analysis

In [ ]:
val = cross_validate(stacking_cv_classifier, X_train, y_train, cv=5, n_jobs=-1)

In [ ]:
#Observa-se  um bom resultado em todas as iterações
val['test_score']

array([0.9613881 , 0.96380134, 0.96201554, 0.96524929, 0.96490974])

In [ ]:
val['test_score'].mean()

0.9634728018701706

In [ ]:
scoring(y_target=y_test,  y_predicted=y_pred, metric='error')

0.036534970841540226

In [ ]:
scoring(y_target=y_test,  y_predicted=y_pred, metric='precision') #VP/VP+FP

0.9693666096044689

In [ ]:
scoring(y_target=y_test,  y_predicted=y_pred, metric='recall') # VP/FN+VP

0.946678398592169

In [ ]:
scoring(y_target=y_test,  y_predicted=y_pred, metric='f1') #precision.recall/precision+recall

0.9578881766381766

## Salvando modelo

In [ ]:
joblib.dump(stacking_cv_classifier, 'modelo_final.pkl')

['modelo_final.pkl']

In [ ]:
loaded_model = joblib.load('modelo_final.pkl')

In [ ]:
pred = loaded_model.predict([X_test.iloc[2]])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but ExtraTreesClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
pred

array([0])

# Stacking - ex2

In [ ]:
url_ex = 'https://raw.githubusercontent.com/alura-cursos/combina-classificadores/main/dados/bank_additional_full_processed.csv'

In [ ]:
df = pd.read_csv(url_ex)

In [ ]:
df.head(1)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,1,0,0,0,1,0,0,0,1,0


In [ ]:
df.info()

In [ ]:
X = df.drop('y', axis=1)
y = df['y']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
params = {'bootstrap': False,
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 6,
 'n_estimators': 200}

In [ ]:
#Classificadores de base
catboost = CatBoostClassifier(verbose=0)
extratrees = ExtraTreesClassifier(**params, random_state=42)
gaussiannb = GaussianNB()

In [ ]:
#Scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)

In [ ]:
#Meta Classifier
logistic = LogisticRegression()

In [ ]:
#Instânciando o Stacking
stack = StackingCVClassifier(classifiers=[catboost,extratrees,gaussiannb],
         meta_classifier=logistic,
         cv=5,
         random_state=42,
         verbose=1,
         n_jobs=-1
)

In [ ]:
#Ajustar modelo aos dados de treino
stack.fit(X_train, y_train)

Fitting 3 classifiers...
Fitting classifier1: catboostclassifier (1/3)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting classifier2: extratreesclassifier (2/3)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   40.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting classifier3: gaussiannb (3/3)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


StackingCVClassifier(classifiers=[<catboost.core.CatBoostClassifier object at 0x7fbefc36bdc0>,
                                  ExtraTreesClassifier(min_samples_split=6,
                                                       n_estimators=200,
                                                       random_state=42),
                                  GaussianNB()],
                     cv=5, meta_classifier=LogisticRegression(), n_jobs=-1,
                     random_state=42, verbose=1)

In [ ]:
#Fazendo previsão  com pipeline
y_pred = stack.predict(X_test)

In [ ]:
stack.score(X_test, y_test)

0.9166059723233795